In [1]:
from napkin import response, request
from pymongo import MongoClient, UpdateOne
import json
import os

CLIENT_NAME = request.path_params.get("CLIENT_NAME")
MONGO_AUTH = os.environ.get("MONGO_AUTH")
body = json.loads(request.data)
if isinstance(body, list) != True:
  body = [body]


In [2]:
from pymongo import MongoClient, UpdateOne

client = MongoClient("mongodb+srv://{}@meusite.hcxhp.mongodb.net/Site?retryWrites=true&w=majority".format(MONGO_AUTH))
db = client.Site

Coll = db[CLIENT_NAME]

In [3]:
ids = map(lambda item: item.get("block").capitalize(), body)

cursor = Coll.find({ "_id": { "$in": list(ids) } })
data = [doc for doc in cursor]

print(data)

[{'_id': 'Metadata', 'logo': 'Eduardo.Viva', 'name': 'Eduardo', 'role': 'UX / UI / Product Designer'}]


In [4]:
result = dict()

for obj in body:
    block = obj.get("block").capitalize()
    current = [x for x in data if x.get("_id") == block]
    result[block] = current[0] if len(current) > 0 else dict()
    del obj["block"]
    result[block].update(obj)

print(result)

{'Metadata': {'_id': 'Metadata', 'logo': 'Eduardo.Viva', 'name': 'Leo ne', 'role': 'UX / UI / Product Designer'}}


In [5]:

operations = []
for key in result:
  document = result[key]
  operations.append(UpdateOne({ "_id": key }, { "$set": document }, upsert=True))
  
if len(operations) > 0:
  res = Coll.bulk_write(operations)
  print("Modified: {}".format(res.modified_count))

Modified: 1
